# [Deploying a Vision Transformer Deep Learning Model with FastAPI in Python](https://pyimagesearch.com/2024/09/23/deploying-a-vision-transformer-deep-learning-model-with-fastapi-in-python/)

Vision Transformers process an image by dividing it into patches, treating each patch as a sequence element, much like words in a sentence. This approach allows Vision Transformers to capture global contextual information across an image.

The strength of Vision Transformers lies in their ability to model relationships between distant parts of an image, which traditional convolutional neural networks (CNNs) might struggle to capture.

---

Uses

- FastAPI
- PyTorch to import and run the model for inference
- DOcker (if you want)
- Curl test (if you want)
- PyTest, which enables to write simple and scalable test cases for Python code

```python

# === testing utils

import pytest
from pyimagesearch.utils import load_model

# Test model loading
def test_load_model():
    model = load_model(MODEL_PATH)
    assert model is not None
    assert isinstance(model, torch.jit.ScriptModule)

# Test image transformations
def test_get_transforms():
    transforms = get_transforms()
    assert transforms is not None

# Test prediction function (partial)
def test_predict():
    model = load_model(MODEL_PATH)
    with open(TEST_IMAGE_PATH, "rb") as f:
        image_bytes = f.read()
    predictions = predict(model, image_bytes)
    # Further validation steps and assertions (not shown)

# === testing main

import logging
import numpy as np
from fastapi.testclient import TestClient
from main import app
from pyimagesearch import config

client = TestClient(app)

TEST_IMAGE_PATH = config.test_image_path

def test_health():
    logging.info("Testing /health endpoint")
    response = client.get("/health")
    logging.debug(f"Response status: {response.status_code}")
    logging.debug(f"Response JSON: {response.json()}")
    assert response.status_code == 200
    assert response.json() == {"message": "ok"}

# === testing inference

def test_infer():
    logging.info("Testing /infer endpoint")
    with open(TEST_IMAGE_PATH, "rb") as f:
        response = client.post(
            "/infer", files={"image": ("test_image.png", f, "image/png")}
        )

    logging.debug(f"Response status: {response.status_code}")
    logging.debug(f"Response JSON: {response.json()}")
    assert response.status_code == 200
    assert isinstance(response.json(), dict)

    # Extract probabilities from the JSON response
    probabilities = response.json()

    # Convert probabilities to a NumPy array and find the index with the highest probability
    probabilities_array = np.array(list(probabilities.values()))
    predicted_label = int(np.argmax(probabilities_array))
    logging.info(f"Predicted label: {predicted_label}")
    
    # Assuming you expect a specific label, you can check it like this
    expected_label = 5  # Replace with the correct label you expect
    assert (
        predicted_label == expected_label
    ), f"Expected {expected_label}, but got {predicted_label}"

```
